In [1]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from get_llm import get_llm  # Wrapper for LangChain around OpenAI API, uses LM Studio


## Chatbot Implementation - Illustration of basic idea of chat history
## Make sure LM Studio runs and suitable model is loaded
## This is a simple demo and can be done with about < 1K token context size

In [2]:
model = get_llm()  # Wrapper for LangChain around OpenAI API
model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000019BD8945050>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000019BD8E80B10>, root_client=<openai.OpenAI object at 0x0000019BD8944510>, root_async_client=<openai.AsyncOpenAI object at 0x0000019BD8E80790>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://localhost:1234/v1')

In [3]:
query = input("User: ")
messages = [
    {
        "role": "system",
        "content": "You are a helpful, friendly assistant"
    },
    {
        "role": "user",
        "content": query
    },
]
response1 = model.invoke(messages)  # model.invoke([HumanMessage(content="What's my name?")])
print(response1.content)

User:  Hi I am Ananth


Hi Ananth! It's lovely to meet you. 😊 

I'm here to help with whatever you need. Just let me know what's on your mind! What can I do for you today?


In [4]:
query = "What is my name?"
messages = [
    {
        "role": "system",
        "content": "You are a helpful, friendly assistant"
    },
    {
        "role": "user",
        "content": query
    },
]
response1 = model.invoke(messages)  # model.invoke([HumanMessage(content="What's my name?")])
print(response1.content)

As an AI, I don't know your name! You haven't told me yet. 😊 

Feel free to tell me your name if you'd like – it would be nice to address you properly!



What's your name?


### Note from the above that the model doesn't remember a previous context, we need to pass explicitly

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Ananth"),
        AIMessage(content="Hello Ananth! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
).content

'Your name is Ananth. 😊 You told me that yourself!'

In [6]:
model.invoke(
    [
        HumanMessage(content="Hi! I'm Ananth"),
        AIMessage(content="Hello Ananth! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
        AIMessage(content='Your name is Ananth!'),
        HumanMessage(content="Address me by my name and answer: What is the capital of India?"),
        
    ]
).content

'Hello Ananth! The capital of India is New Delhi.'

### Basic Idea: When we pass history explicitly the model can remember the context - so keep a small memory and pass everytime to the model

## Message History

### We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

### We may also want to support multiple sessions. So we can define a session id and use it to identify the session.
### We need to get a reference to a message history object for the given session
### Write a function get_session_history that takes a session id and returns a message history 
### Also define RunnableWithMessageHistory with parameters model and get_session_history function
###  This session_id is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

In [7]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}  # maps a session id to a history object


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = {"configurable": {"session_id": "abc2"}}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Ananth")],
    config=config,
)

response.content

"Hi Ananth! It's great to meet you. 😊 \n\nWhat can I do for you today? Do you have any questions, need help with something, or just want to chat?"

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"Your name is Ananth! You told me that when we first started chatting. 😄\n\n\n\nIt's a pleasure to be talking with you, Ananth."

In [11]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"As an AI, I don't know your name. You haven't told me! 😊 \n\nYou can tell me if you like.\n\n\n\n"

In [12]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'You know what? You are absolutely right to ask again! My apologies. I seem to be having a little trouble remembering. \n\nYour name is **Ananth**.\n\n\n\nI am still under development and learning to remember things better. Thanks for pointing out my mistake!'

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Who is the founder of Adobe?")],
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Does it make Photoshop?")],
    config=config,
)

response.content